<ol>
    <li>First, Get the Input data, and classify them according to their intents</li>
    <li>Then From that classifiar, we will predict the intent of user's query</li>
    <li><strike>After getting the intent of user, we will apply the logic of data driven bot</li>
    <li>According to intent, and the question, apply the TF-IDF Logic and find similarity between a particular answer and question</li>
    <li>One with highest similarity will be given as response to the user</strike></li>
</ol>

### Classes
#### 0. Admission Enquiry
#### 1. Contact Info
#### 2. About Courses and Programms
#### 3. Campus Enquiry

In [1]:
import nltk
import numpy as np
import re
import string
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [2]:
dataset = [["Is Admission Open?", 0], \
           ["Is there any seat available in this college?", 0], \
           ["How many seats are available in this college?", 0], \
           ["Is there any seat available in Information Technology Department?", 0], \
           ["I want to take admission in Dental College, is there any seat available?", 0], \
           ["Can i take admission through GUJCET in this college?", 0], \
           ["What was the last year cut off for the admission?", 0], \
           ["I want Admission Cell email id for contact", 1], \
           ["I want admin contact number", 1], \
           ["I want admin department contact number", 1], \
           ["I want hostel contact information", 1], \
           ["I want toll free contact number", 1], \
           ["Give or send me your website link", 1], \
           ["What are the available programms in this university ?", 2], \
           ["What are the technical courses this university offer?", 2], \
           ["What are the medical courses this university offer?", 2], \
           ["What are the courses in the Engineering Department?", 2], \
           ["Is there any Fest being organized in the College?", 2], \
           ["Is there any special international program in the college?", 2], \
           ["I want to know about the college", 3], \
           ["I want to know about the parking facility in the college", 3], \
           ["What about Cafeteria,Is there any cafetaria in college or campus?", 3], \
           ["Is there any library in here?", 3], \
           ["Is there any Cafeteria in college?", 3], \
           ["What about Canteen in the college?", 3], \
           ["What about garden in the college?", 3], \
           ["How many books are there in library?", 3] \
          ]

### Training the Data

In [3]:
lemma = WordNetLemmatizer()

In [4]:
def LemWords(words):
    return ' '.join(lemma.lemmatize(word) for word in words)

In [5]:
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

In [6]:
def preprocessSent(sent):
    sent = sent.lower()
    sent = sent.translate(remove_punct_dict)
    words = nltk.word_tokenize(sent)
    sent = LemWords(words)
    return sent

In [7]:
dataset = [[preprocessSent(data[0]), data[1]] for data in dataset]

In [8]:
dataset

[['is admission open', 0],
 ['is there any seat available in this college', 0],
 ['how many seat are available in this college', 0],
 ['is there any seat available in information technology department', 0],
 ['i want to take admission in dental college is there any seat available', 0],
 ['can i take admission through gujcet in this college', 0],
 ['what wa the last year cut off for the admission', 0],
 ['i want admission cell email id for contact', 1],
 ['i want admin contact number', 1],
 ['i want admin department contact number', 1],
 ['i want hostel contact information', 1],
 ['i want toll free contact number', 1],
 ['give or send me your website link', 1],
 ['what are the available programms in this university', 2],
 ['what are the technical course this university offer', 2],
 ['what are the medical course this university offer', 2],
 ['what are the course in the engineering department', 2],
 ['is there any fest being organized in the college', 2],
 ['is there any special internati

In [9]:
df = pd.DataFrame(dataset, columns = ['text', 'intent'])

In [10]:
X = df['text']
y = df['intent']

In [11]:
tfidfVectorizer = TfidfVectorizer()

In [12]:
X = tfidfVectorizer.fit_transform(X)

In [13]:
classifier = LinearSVC(max_iter = 1000, C = 0.1)

In [14]:
classifier.fit(X, y)

LinearSVC(C=0.1, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

### From the Predicted class, Give response using NER

In [15]:
from nltk.chunk import tree2conlltags
from nltk.chunk import ne_chunk
from nltk import pos_tag, sent_tokenize, word_tokenize

In [16]:
responses = {0 : """Admission Desk is open till 20th July for Students like You😁, You can ask them questions anytime. 
There are some seats available in Engineering, Science, MCA and Dental Department.  
You can take admission either via GUJCET Exam or JEE. 
Admission Process can be done either via Online Mode or Offline Mode.
""", \
1 : """You can reach Admission Cell, Email ID abc@xyz.com.
You can reach Admission Cell, Mobile No 00000000000.
Contact us on Toll Free Mobile, 11111111111.
To Contact the Hostel Cell, Email on cdf@xyz.com.
Sorry, We can't give you Admin Details😅.
""", \
2 : """This College provides various courses in Engineering like , Information Technology, Computer Science, Mechanical, Civil, Chemical.
There is a Medical Science Department, which provides various types of Biology Courses.
This College provides Special Global Student Exchange Program. 
Every Year, This College organizes Cultural Fest named 'Felicific' having a week of full of Excitement and Joy.
""", \
3 : """ College Cafeteria is one of the largest in the city, and serves pure and hygenic food.
Our Library has over 100000 books.
College has parking lot for over 1000 vehicles.
There are 3 Gardens and 1 Main Garden in this College.
This College has a very vast Campus Infrastructor, containing Gardens, Library, Cricket and Football Ground etc.
""", \
}

In [17]:
entities = []
for i in range(len(responses.keys())):
    temp = []
    for sent in sent_tokenize(responses[i]):
        for chunk in ne_chunk(pos_tag(word_tokenize(sent))):
            # print(tree2conlltags(ne_chunk(pos_tag(word_tokenize(sent)))))
            if hasattr(chunk, 'label'):
                temp.append([chunk[0][0], sent])
                # print(chunk.label(), ' '.join(c[0] for c in chunk))
                                          
    entities.append(temp)

In [18]:
# entities

In [19]:
def LemTokens(tokens):
    return [lemma.lemmatize(token) for token in tokens]

In [20]:
def Normalize(text): #Remove punctuation, then word tokenize, Then Lemmatize Each Token
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [21]:
def response(user_query): 
    robo_response = ''
    # Predict the intent of the User_query
    X_test = tfidfVectorizer.transform([user_query])
    prediction = classifier.predict(X_test)
    entity = entities[prediction[0]]
    sents = []
    # Try to find if the user_query contains any of the entity of that intent class or not
    for item in entity:
        if item[0].lower() in user_query.lower():
            sents.append(item[1])
            # robo_response += item[1] + '\n'
    
    # If there is no entity in the user_query according to the prediction class
    if len(sents) == 0:
        resp = "Sorry, I can't understand you.\n😅Can you please rephrase the query Or For Any "
        if prediction[0] == 0:
            resp += "Admission Query"
        elif prediction[0] == 1:
            resp += "Contact Info"
        elif prediction[0] == 2:
            resp += "Courses and Programms related Query"
        elif prediction[0] == 3:
            resp += "Campus related Query"
        resp += ", You can Check our website for more information"
        return resp
    
    sents.append(user_query)
    tfidf = TfidfVectorizer(tokenizer = Normalize, stop_words='english')
    vect = tfidf.fit_transform(sents)
    vals = cosine_similarity(vect[-1], vect)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req = flat[-2]
    if req!=0:
        robo_response += sents[idx]
    # robo_response += "\n\n😄Please Check our website for more information."
    return robo_response

In [22]:
test = "College?"

In [23]:
response(test)

'College has parking lot for over 1000 vehicles.'

In [24]:
import requests
import json

class telegram_bot():
    def __init__(self):
        self.token = "1365657286:AAF5fh9IaRG-dBmz10vp5mJRDpSGit6L0nE"    #write your token here!  
        self.url = f"https://api.telegram.org/bot{self.token}"

    def get_updates(self,offset=None):
        url = self.url+"/getUpdates?timeout=100"
        if offset:
            url = url+f"&offset={offset+1}"
        url_info = requests.get(url)
        return json.loads(url_info.content)

    def send_message(self,msg,chat_id):
        url = self.url + f"/sendMessage?chat_id={chat_id}&text={msg}"
        if msg is not None:
            requests.get(url)

    def grab_token(self):
        return tokens

In [25]:
tbot = telegram_bot()

update_id = None

def make_reply(msg):
    if msg is not None:
        if 'website' in msg.lower():
            return "Our site link is, 'www.ddu.ac.in' ."
        if msg.lower() == 'bye' or msg.lower() == 'good bye':
            return 'Good Bye'
        if len(word_tokenize(msg)) == 1:
            return 'Hello\nWelcome to the unofficial Assistant of DDU College😄\nWhat can I do for You?'    
    reply = response(msg)
    return reply
flag = 0
while True:
    print("...")
    updates = tbot.get_updates(offset=update_id)
    updates = updates['result']
    print(updates)
    if updates:
        for item in updates:
            update_id = item["update_id"]
            print(update_id)
            try:
                message = item["message"]["text"]
                print(message)
            except:
                message = None
            from_ = item["message"]["from"]["id"]
            print(from_)

            reply = make_reply(message)
            tbot.send_message(reply,from_)


...
[{'update_id': 312269594, 'message': {'message_id': 67, 'from': {'id': 1143427314, 'is_bot': False, 'first_name': 'Jaimish', 'language_code': 'en'}, 'chat': {'id': 1143427314, 'first_name': 'Jaimish', 'type': 'private'}, 'date': 1594625317, 'text': '/start', 'entities': [{'offset': 0, 'length': 6, 'type': 'bot_command'}]}}]
312269594
/start
1143427314
...
[{'update_id': 312269595, 'message': {'message_id': 69, 'from': {'id': 1143427314, 'is_bot': False, 'first_name': 'Jaimish', 'language_code': 'en'}, 'chat': {'id': 1143427314, 'first_name': 'Jaimish', 'type': 'private'}, 'date': 1594625393, 'text': 'I want to take admission in engineering, is there any seat available?'}}]
312269595
I want to take admission in engineering, is there any seat available?
1143427314
...
[{'update_id': 312269596, 'message': {'message_id': 71, 'from': {'id': 1143427314, 'is_bot': False, 'first_name': 'Jaimish', 'language_code': 'en'}, 'chat': {'id': 1143427314, 'first_name': 'Jaimish', 'type': 'private'}

KeyboardInterrupt: 